In [1]:
# NEURAL NETWORK FOR FRUITS DETECTOR

In [2]:
# Pertama, import dulu tensorflow.
import tensorflow as tf
import os

In [3]:
# Kedua, setup base directory dari data-data training. Pilih folder /fruits-360/Training. 
base_dir = './fruits-360/fruits-360_dataset/fruits-360/Training'

In [4]:
# Set ukuran gambar dan banyaknya batch.
# Keduanya tidak boleh berlebihan atau kurang dikarenakan
# Berpengaruh pada keakuratan training model.
IMAGE_SIZE = 224
BATCH_SIZE = 64

# Init variabel datagen untuk basis generate data ai,
# Lalu tambahkan setting yang sesuai dengan code di bawah.
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      validation_split=0.1,
      fill_mode='nearest')

# Setup generator training ai sesuai ukuran gambar
# dan batch yang sebelumnya sudah ditentukan.
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='training')

# Sama seperti train_generator untuk validasi 
# training ai. Setiap ai yang ditraining perlu
# divalidasi keakuratannya.
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE, 
    subset='validation')

Found 60955 images belonging to 131 classes.
Found 6737 images belonging to 131 classes.


In [5]:
# Coba print value shape dari gambar dan juga value label.
for image_batch, label_batch in train_generator:
  break
image_batch.shape, label_batch.shape

((64, 224, 224, 3), (64, 131))

In [6]:
# Print isi class dari setup variabel train_generator
print (train_generator.class_indices)

# Buat txt untuk labels. Labels ini nanti akan di-export
# bersamaan dengan isi training model untuk identifikasi
# object apa (dalam kasus ini buah) yang ai dapatkan.
labels = '\n'.join(sorted(train_generator.class_indices.keys()))

# Export labels dalam format .txt.
with open('labels.txt', 'w') as f:
  f.write(labels)

{'Apple Braeburn': 0, 'Apple Crimson Snow': 1, 'Apple Golden 1': 2, 'Apple Golden 2': 3, 'Apple Golden 3': 4, 'Apple Granny Smith': 5, 'Apple Pink Lady': 6, 'Apple Red 1': 7, 'Apple Red 2': 8, 'Apple Red 3': 9, 'Apple Red Delicious': 10, 'Apple Red Yellow 1': 11, 'Apple Red Yellow 2': 12, 'Apricot': 13, 'Avocado': 14, 'Avocado ripe': 15, 'Banana': 16, 'Banana Lady Finger': 17, 'Banana Red': 18, 'Beetroot': 19, 'Blueberry': 20, 'Cactus fruit': 21, 'Cantaloupe 1': 22, 'Cantaloupe 2': 23, 'Carambula': 24, 'Cauliflower': 25, 'Cherry 1': 26, 'Cherry 2': 27, 'Cherry Rainier': 28, 'Cherry Wax Black': 29, 'Cherry Wax Red': 30, 'Cherry Wax Yellow': 31, 'Chestnut': 32, 'Clementine': 33, 'Cocos': 34, 'Corn': 35, 'Corn Husk': 36, 'Cucumber Ripe': 37, 'Cucumber Ripe 2': 38, 'Dates': 39, 'Eggplant': 40, 'Fig': 41, 'Ginger Root': 42, 'Granadilla': 43, 'Grape Blue': 44, 'Grape Pink': 45, 'Grape White': 46, 'Grape White 2': 47, 'Grape White 3': 48, 'Grape White 4': 49, 'Grapefruit Pink': 50, 'Grapefrui

In [7]:
# Setup size untuk ukuran gambar di model ai gambar.
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

# Setup base model untuk traning base ai gambar.
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                              include_top=False, 
                                              weights='imagenet')

In [8]:
# Setup trainable ke false. Belum ditesting lebih jauh apabila
# di set menjadi true, dan ini bisa mempengaruhi hasil training
# ai tetapi belum diketahui seberapa jauh mempengaruhi hasil
# akurasi.
base_model.trainable = False

In [9]:
# Tambah dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

# Tambah specifically untuk cnn (convolutional neural network)
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

# Mulai modelling menggunakan cnn (convolutional neural network)

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(131, activation = "softmax"))

In [10]:
# Setup model cnn tadi menggunakan tensorflow keras.
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2,2), strides=(2,2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(131, activation='softmax')
])

In [11]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(131, activation='softmax')
])

In [12]:
# Setup compile untuk training model.
model.compile(optimizer=tf.keras.optimizers.Adam(), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [13]:
# Set epochs (epoch merupakan jumlah total iterasi 
# semua data training dalam one cycle untuk melatih model ai)
epochs = 10

# Mulai compile training model ai. Lalu hasilnya
# masuk ke variable history.
history = model.fit(train_generator, 
                    epochs=epochs,
                    steps_per_epoch=50,
                    )

Epoch 1/10


50/50 [==============================] - 29s 553ms/step - loss: 4.4243 - accuracy: 0.0721
Epoch 2/10
50/50 [==============================] - 30s 601ms/step - loss: 3.3984 - accuracy: 0.2319
Epoch 3/10
50/50 [==============================] - 30s 590ms/step - loss: 2.4814 - accuracy: 0.3966
Epoch 4/10
50/50 [==============================] - 29s 576ms/step - loss: 1.7263 - accuracy: 0.5500
Epoch 5/10
50/50 [==============================] - 28s 563ms/step - loss: 1.3017 - accuracy: 0.6416
Epoch 6/10
50/50 [==============================] - 29s 569ms/step - loss: 1.0438 - accuracy: 0.7016
Epoch 7/10
50/50 [==============================] - 28s 551ms/step - loss: 0.8698 - accuracy: 0.7477
Epoch 8/10
50/50 [==============================] - 28s 549ms/step - loss: 0.8135 - accuracy: 0.7656
Epoch 9/10
50/50 [==============================] - 27s 545ms/step - loss: 0.6952 - accuracy: 0.7962
Epoch 10/10
50/50 [==============================] - 27s 531ms/step - loss: 0.6049 - accu

In [14]:
# Lalu simpan hasilnya dengan nama file ini.
model.save('fruitsnn.h5')

C:\Users\dhimasdewanto\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
# Supaya flutter bisa baca model tensorflow, hasil training ai ini
# harus diubah terlebih dahulu ke tipe data .tflite. 
# Berikut setup model2 (model baru) untuk konversinya.
model2=tf.keras.models.load_model('fruitsnn.h5', compile=False)

In [16]:
# Mulai konversi dari .h5 ke .tflite.
converter = tf.lite.TFLiteConverter.from_keras_model(model2)
tflite_model = converter.convert()

# Export file .tflite.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\DHIMAS~1\AppData\Local\Temp\tmpz71lvgtu\assets


INFO:tensorflow:Assets written to: C:\Users\DHIMAS~1\AppData\Local\Temp\tmpz71lvgtu\assets
